In [ ]:
import pandas as pd
from airtable import airtable

import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
sys.path.pop()

import dotenv
import os
dotenv.load_dotenv()
at_api = os.environ["AIRTABLE_API_TOKEN"]
at_base = os.environ["AIRTABLE_DEVREL_BASE_ID"]

columns = ['Name','Alignment','Block Explorer','Announcement Date', 'Mainnet Launch Date'
           ,'L1 Standard Bridge Contract', 'L1 NFT Bridge Contract', 'L1 BatchInbox "to address"', 'L1 BatchInbox "from address"']

In [ ]:
at = airtable.Airtable(at_base, at_api)
data = at.get('OP Products')
df = pd.json_normalize(data, record_path='records')
# Rename all columns that start with 'fields.'
df.rename(columns=lambda x: x.replace('fields.', ''), inplace=True)
df = df[columns]

In [ ]:
# Filter the DataFrame
filtered_df = df[df['L1 Standard Bridge Contract'].notnull() & df['L1 Standard Bridge Contract'].str.startswith('0x')\
                 & df['L1 BatchInbox "to address"'].notnull() & df['L1 BatchInbox "to address"'].str.startswith('0x')]
filtered_df.reset_index(inplace=True, drop=True)
display(filtered_df)

In [ ]:
# Post to Dune API
d.write_dune_api_from_pandas(filtered_df, 'op_stack_chain_metadata','Metadata about OP Stack Chains (i.e. bridge contracts, submitter contracts, name)')